# Process RR_peaks to metrics

**The following notebook** 
* Reads the RR peaks file
* Derives metrics from 5 min intervals
* Joins clinical indicator file with metrics on patient id 
* Saves the file

In [1]:
import pandas as pd
import h5py
import helper

read_path = "our_processing/Nabian_2018_derrived_data.h5"
pickle_path = 'our_processing/Nabian_2025_10_02.pkl'

Read the RR_Peaks file ["our_processing\ML_Data.h5"]("our_processing\ML_Data.h5")

In [ ]:


def read_data(file_path, group_name, signal_name= "r_intervals"):
    with h5py.File(file_path, "r") as hdf:
        group = hdf[group_name]
        data = group[signal_name][()]
    return data

def get_page_names(file_path):
    with h5py.File(file_path, "r") as hdf:
        page_names = list(hdf.keys())
    return page_names

page_names = get_page_names(read_path)
print(page_names)

patient_arrays = {group_name: read_data(read_path, group_name) for group_name in page_names}

patient_arrays

{'19070921': array([764., 772., 780., ..., 708., 696., 704.]),
 '19072205': array([824., 772., 776., ..., 820., 800., 804.]),
 '19072214': array([1268.,  632.,  644., ..., 1332., 1576., 1812.]),
 '19072938': array([684., 700., 724., ..., 688., 676., 448.]),
 '19072939': array([732., 720., 668., ..., 656., 664., 556.]),
 '19072940': array([ 668.,  656.,  656., ..., 1016.,  472.,  912.]),
 '19080106': array([892., 888., 856., ..., 816., 816., 560.]),
 '19080715': array([1100.,  948.,  964., ..., 1228., 1044., 1440.]),
 '19081506': array([ 884.,  832.,  824., ..., 1016.,  520., 1608.]),
 '19082406': array([ 692.,  680.,  668., ..., 1780.,  792., 1276.]),
 '19090308': array([624., 624., 620., ..., 764., 768., 748.]),
 '19090320': array([708., 712., 684., ..., 636., 644., 620.]),
 '19101607': array([1508.,  956.,  972., ...,  604.,  848., 1388.]),
 '19101619': array([1356.,  952.,  944., ...,  740.,  752.,  748.]),
 '19102102': array([784., 804., 784., ..., 648., 648., 644.]),
 '19102103': 

['19070921', '19072205', '19072214', '19072938', '19072939', '19072940', '19080106', '19080715', '19081506', '19082406', '19090308', '19090320', '19101607', '19101619', '19102102', '19102103', '19102524', '19102622', '19112609', '19120302', '19120323', '19120704', '19120723', '19121303', '19121735', '20010826', '20010827', '20011712', '20050628', '20052606', '20061729', '20092226', '20092535', '20101424', '20101822', '20102029', '20120116', '20120922', '20121033', '20121716', '20121718', '20122932', '20123017']


Calculate the metrics for each patient.

In [3]:
metrics_df = pd.DataFrame()

for id, signal in patient_arrays.items():
    individual_metrics = helper.patients_metrics(signal)
    individual_metrics['id'] = id
    print(id, "processed")
    metrics_df = pd.concat([metrics_df, individual_metrics], ignore_index=True)

19070921 processed
19072205 processed
19072214 processed
19072938 processed


IndexError: index 0 is out of bounds for axis 0 with size 0

Read the clinical indicators

In [ ]:
clin_indic = helper.df_from_excel('actionable_data/Clinical indicators.xlsx')
clin_indic.rename(columns={'number': 'id'}, inplace=True)
clin_indic['id'] = clin_indic['id'].astype(str)
display(clin_indic.columns)

Inner merge the clinical indicators and processing results

In [ ]:
metrics_df["id"].astype(str)
merged_data = metrics_df.merge(clin_indic, on='id', how='inner')

merged_data.head()

In [ ]:
merged_data.to_pickle(pickle_path)
merged_data.to_csv(pickle_path.replace(".pkl", '.csv'), index=False)